In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 如何使用 LangChain 🦜️🔗 Bigquery 數據載入器

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/orchestration/langchain/langchain_bigquery_data_loader.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/orchestration/langchain/langchain_bigquery_data_loader.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在 GitHub 上查看
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/orchestration/langchain/langchain_bigquery_data_loader.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在 Vertex AI 工作台上開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Karl Weinmeister](https://github.com/kweinmeister) |


## 目標

此筆記本提供了一個關於如何使用 [Langchain](https://langchain.com/) 及 Langchain 的 [BigQuery 資料載入器](https://python.langchain.com/docs/integrations/document_loaders/google_bigquery) 的介紹性理解。此筆記本包含 3 個步驟：

1. 使用 LangChain 查詢 Vertex AI LLM
2. 使用 LangChain BigQuery 資料載入器
3. 開發使用資料載入器的鏈


### 成本

本教學使用 Google Cloud 的計費元件：

- Vertex AI
- BigQuery

深入了解 [Vertex AI 定價](https://cloud.google.com/vertex-ai/pricing)、[BigQuery 定價](https://cloud.google.com/bigquery/pricing)，
並使用 [定價計算器](https://cloud.google.com/products/calculator/)
根據你的預期使用量產生成本估算。


In [ ]:
# Install Vertex AI LLM SDK, BigQuery library, and langchain
!pip install google-cloud-aiplatform google-cloud-bigquery langchain==0.0.332 --upgrade --user

**Colab 獨有：** 取消以下Cell註解以重新啟動核心，或使用按鈕重新啟動核心。對於 Vertex AI Workbench，你可以使用頂端的按鈕重新啟動終端機。


In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 驗證你的筆記本環境

- 如果你使用 **Colab** 來執行此筆記本，請取消以下單元格註解並繼續。
- 如果你使用 **Vertex AI Workbench** ，請查看此處的設定說明 [網址](https://github.com/doggy8088/generative-ai/tree/main/setup-env)。


In [ ]:
# from google.colab import auth as google_auth
# google_auth.authenticate_user()

### 匯入函式庫


**僅 Colab：** 取消註解以下Cell以初始化 Vertex AI SDK。對於 Vertex AI Workbench，不需要執行此作業。


In [ ]:
# import vertexai

# PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
# vertexai.init(project=PROJECT_ID, location="us-central1")

In [ ]:
import google.cloud.bigquery as bq
import langchain
from google.cloud import aiplatform
from langchain.llms import VertexAI
from langchain.document_loaders import BigQueryLoader
from langchain.prompts import PromptTemplate
from langchain.schema import format_document

# Print LangChain and Vertex AI versions
print(f"LangChain version: {langchain.__version__}")
print(f"Vertex AI SDK version: {aiplatform.__version__}")

## 使用 Vertex AI 基礎模型與 LangChain


讓我們從頭開始，並在過程中學習一些 BigQuery 的知識。我們將定義一個 LangChain LLM 模型。我們將使用 [文字基礎模型](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text) 並將 `溫度` 設定為 0 以獲得一致的結果。


In [ ]:
llm = VertexAI(model_name="text-bison@001", temperature=0)

llm("What's BigQuery?")

## 使用資料載入器

現在我們來學習如何使用文件載入器。我們將使用一個虛構的電子商務服飾網站 [TheLook](https://console.cloud.google.com/marketplace/product/bigquery-public-data/thelook-ecommerce) 中的資料，此網站可作為 BigQuery 公開資料集取得。

我們的首要目標是了解資料集中的資料表。讓我們查詢這個資料集中的架構，以擷取資料定義語言 (DDL)。DDL 用於建立和變更資料表，可以告訴我們各個欄和其類型的資訊。

我們的查詢會擷取各個資料表的資料表名稱和 DDL。接著建立一個 [資料載入器](https://api.python.langchain.com/en/latest/document_loaders/langchain.document_loaders.bigquery.BigQueryLoader.html)，指定資料表名稱為 metadata 欄，而 DDL 則為內容。


In [ ]:
# Define our query
query = f"""
SELECT table_name, ddl
FROM `bigquery-public-data.thelook_ecommerce.INFORMATION_SCHEMA.TABLES`
WHERE table_type = 'BASE TABLE'
ORDER BY table_name;
"""

# Load the data
loader = BigQueryLoader(
    query, metadata_columns="table_name", page_content_columns="ddl"
)
data = loader.load()

## 編寫我們的首個鏈

載入文件後，讓我們開始使用它們。

我們的目標是了解在日本即將展開的行銷活動中，我們想要鎖定的客戶。我們將使用 [程式碼產生模型](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/code-generation) 協助我們建立查詢。

我們將建立一個簡單的鏈，將所有表格的相關資料「填入](https://python.langchain.com/docs/use_cases/summarization#option-1-stuff)」一個提示中。對於表格數量更多的較大型資料集，將需要更精密的鏈結方式。這是因為每個提示都有長度限制，也就是說，一個相關視窗。

例如，你可以將每個個別表格內容重點壓縮成較小的文件，然後使用 [對應簡約](https://python.langchain.com/docs/use_cases/summarization#option-2-map-reduce) 方法進行摘要。或者，你可以逐一檢視每個表格，同時 [優化](https://python.langchain.com/docs/use_cases/summarization#option-3-refine) 你的查詢。

以下是使用方法。我們將使用 [LangChain 表達式語言](https://python.langchain.com/docs/expression_language/) (LCEL) 搭配 3 個步驟來定義鏈：

1. 我們將每個文件中的 page_content (記得，那是每個表格的 DDL) 合併到稱為 content 的字串。
1. 建立一個提示來找出最有價值的客戶，傳入 content，合併的表格相關資料集。
1. 將提示傳遞給 LLM。


In [ ]:
# Use code generation model
llm = VertexAI(model_name="code-bison@latest", max_output_tokens=2048)

# Define the chain
chain = (
    {
        "content": lambda docs: "\n\n".join(
            format_document(doc, PromptTemplate.from_template("{page_content}"))
            for doc in docs
        )
    }
    | PromptTemplate.from_template(
        "Suggest a GoogleSQL query that will help me identify my most valuable customers located in Japan:\n\n{content}"
    )
    | llm
)

# Invoke the chain with the documents, and remove code backticks
result = chain.invoke(data).strip("```")
print(result)

讓我們現在來試試我們的查詢，看看它回傳了什麼！


In [ ]:
client = bq.Client()
client.query(result).result().to_dataframe()

恭喜，你現在已了解如何將你的 BigQuery 資料整合到 LLM 解決方案中！ 🎉
